In [29]:
# Import necessary functions
import os
import aryn_sdk
from aryn_sdk.partition import partition_file, draw_with_boxes, convert_image_element
import json
import pdf2image
#from openai import OpenAI
from pydantic import BaseModel
#from openai.lib._parsing import type_to_response_format_param as pydantic_to_response_format
#from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
# Set your secrets in the colab notebook. Navigate to the left pane
# and choose the key option to set your keys. Make sure to enable Notebook access

# Visit https://www.aryn.ai/get-started to get a key.
aryn_api_key = os.getenv('aryn_API_KEY')
# Visit https://platform.openai.com/api-keys to get a key.
openai_api_key = os.getenv('anthropic_API_KEY')

In [41]:
file_name = 'syllabus206.pdf'
file = open(file_name, 'rb')

#show the pdf
#pdf2image.convert_from_path(file_name)[0]

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [34]:
## Make a call to the partitioning service and set extract_images to true.
partitioned_file = partition_file(file, aryn_api_key, extract_images=True, extract_table_structure=True, use_ocr=True)

# show the pdf with bounding boxes
#draw_with_boxes(file_name, partitioned_file)[0]

ArynPartitioner: T+   0.00: Server version aryn-partitioner-0.20241226.161207 Model version 1.4
ArynPartitioner: T+   0.00: Received request with aryn_call_id=7b0434b0-3e3d-45ae-8bcb-c65c81b65992
ArynPartitioner: T+   0.00: Waiting for scheduling
ArynPartitioner: T+   2.42: Preprocessing document
ArynPartitioner: T+   2.42: Done preprocessing document
ArynPartitioner: T+   2.72: Completed work on page 1


In [39]:
## Walk over the image elements in the returned JSON and show the first one
print(partitioned_file)
images = [e for e in partitioned_file['elements']]
print(images)

{'status': ['Incremental status will be shown here during execution.', "Until you get a line that matches '  ]\n', you can convert the partial", 'output to a json document by appending \'""]}\' to the partial output.', '', 'T+   0.00: Server version aryn-partitioner-0.20241226.161207 Model version 1.4', 'T+   0.00: Received request with aryn_call_id=7b0434b0-3e3d-45ae-8bcb-c65c81b65992', 'T+   0.00: Waiting for scheduling', 'T+   2.42: Preprocessing document', 'T+   2.42: Done preprocessing document', 'T+   2.72: Completed work on page 1', ''], 'elements': []}
[]


In [ ]:
# Pydantic models used to form response_format parameter to anthropic
class description(BaseModel):
    image_title: str
    image_description: str

class diagram_response(BaseModel):
    steps: list[description]

In [ ]:
diagram_response_schema = diagram_response.model_json_schema()

tools = [
    {
        "name": "build_description_result",
        "description": "build the description object",
        "input_schema": diagram_response_schema
    }
]

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=openai_api_key)
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": jpeg_img,
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this diagram step by step."
                }
            ],
        }
    ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_description_result"}

)
print(message)


In [ ]:
print(json.dumps(message.content[0].input, indent=4))